## 1.中文分词
### https://github.com/fxsjy/jieba

In [1]:
import jieba

seg_list = jieba.cut("武汉加油！中国加油！没有一个冬天不可逾越，没有一个春天不会来临。", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.685 seconds.
Prefix dict has been built successfully.


Default Mode: 武汉/ 加油/ ！/ 中国/ 加油/ ！/ 没有/ 一个/ 冬天/ 不可逾越/ ，/ 没有/ 一个/ 春天/ 不会/ 来临/ 。


## 2.词频向量化
### CountVectorizer 类会将文本中的词语转换为词频矩阵
### https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [30]:
from sklearn.feature_extraction.text import CountVectorizer


corpus = ['疫情结束，一起去武汉看樱花！',
          '可怕！新冠疫情从武汉蔓延！',
          '军运会在武汉举行。',
          '武汉加油！',
        ]

tokenzied_corpus = [' '.join(jieba.cut(sentence, cut_all=False)) for sentence in corpus]

vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(tokenzied_corpus)
feature_name = vectorizer.get_feature_names()

print (X)
print (feature_name)
for w in X.toarray():
    print('\t'.join([str(round(x, 2)) for x in w]))

  (0, 8)	1
  (0, 9)	1
  (0, 0)	1
  (0, 7)	1
  (0, 6)	1
  (1, 8)	1
  (1, 7)	1
  (1, 4)	1
  (1, 5)	1
  (1, 10)	1
  (2, 7)	1
  (2, 2)	1
  (2, 1)	1
  (3, 7)	1
  (3, 3)	1
['一起', '举行', '军运会', '加油', '可怕', '新冠', '樱花', '武汉', '疫情', '结束', '蔓延']
1	0	0	0	0	0	1	1	1	1	0
0	0	0	0	1	1	0	1	1	0	1
0	1	1	0	0	0	0	1	0	0	0
0	0	0	1	0	0	0	1	0	0	0


## 3.TF-IDF处理

### 然而有些词在文本中尽管词频高，但是并不重要，这个时候就可以用TF-IDF技术。某一特定文件内的高词语频率，以及该词语在整个文件集合中的低文件频率，可以产生出高权重的TF-IDF。因此，TF-IDF倾向于过滤掉常见的词语，保留重要的词语。   
### https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.pipeline import Pipeline
import numpy as np


pipe = Pipeline([('count', CountVectorizer()),
                 ('tfid', TfidfTransformer())]).fit(tokenzied_corpus)

In [32]:
featrure = pipe.transform(tokenzied_corpus)

In [33]:
weight = featrure.toarray()
print(pipe['count'].get_feature_names())
print()
idfs = pipe['tfid'].idf_
print('\t'.join([str(round(x, 2)) for x in idfs]))
print()
for w in weight:
    print('\t'.join([str(round(x, 2)) for x in w]))

#weight就是模型的输入，它的大小(m,n),m是文本个数，n是词库的大小

['一起', '举行', '军运会', '加油', '可怕', '新冠', '樱花', '武汉', '疫情', '结束', '蔓延']

1.92	1.92	1.92	1.92	1.92	1.92	1.92	1.0	1.51	1.92	1.92

0.51	0.0	0.0	0.0	0.0	0.0	0.51	0.26	0.4	0.51	0.0
0.0	0.0	0.0	0.0	0.51	0.51	0.0	0.26	0.4	0.0	0.51
0.0	0.66	0.66	0.0	0.0	0.0	0.0	0.35	0.0	0.0	0.0
0.0	0.0	0.0	0.89	0.0	0.0	0.0	0.46	0.0	0.0	0.0


## 4. xgboost模型

### https://xgboost.readthedocs.io/en/latest/index.html

In [14]:
import re
import numpy as np
import pandas as pd
import xgboost as xgb

In [15]:
with open('./data/baidu_stopwords.txt', 'r') as f:
    stop_words = f.read().split('\n')

stop_words[:10]

['--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according']

In [16]:
def cut_and_clean(text):
    cuted_text = ' '.join([x for x in jieba.lcut(text) if x not in stop_words])
    clean_text = re.sub('([\.，。、“”‘ ’？\?:#：【】\+!！《 》「」『』\{\}\|])', ' ', cuted_text)
    clean_text = re.sub('\s{2,}', ' ', clean_text)
    clean_text = clean_text.lower()
    return clean_text.strip()
text = '武汉的樱花很漂亮！'
cut_and_clean(text)

'武汉 樱花 很漂亮'

In [17]:
train_df = pd.read_csv('./processed_data/train.csv', encoding='utf-8')
train_df['processed_text'] = train_df['text'].apply(cut_and_clean)
train_df.head()

,text,label,processed_text
0,肺炎疫情,0,肺炎 疫情
1,【西方记者挑事提问，WHO总干事用事实力挺中国】12日，世界卫生组织大会上，有西方记者挑事提...,0,西方 记者 挑事 提问 who 总干事 用事 实力 挺 中国 12 日 世界卫生组织 大会 ...
2,人数终于 始呈下降趋势。这个春节有喜有忧，但事情终有完结的那一天。中午出去买了一次蔬菜，过了...,0,人数 终于 始 呈 下降 趋势 春节 有喜有忧 事情 终 完结 中午 买 蔬菜 六道 关卡 ...
3,【抗病毒小刘上线】今天有好几个朋友来问我戴没戴口罩要不要寄口罩给我，超级感动，小刘除了被吓到...,0,抗病毒 小 刘 上线 好几个 朋友 问 戴 没 戴 口罩 寄 口罩 超级 感动 小 刘 惊慌...
4,潜伏期传染，后期会不会更惊讶O最新疫情地图出炉，这传播速度有让你觉得惊讶吗,0,潜伏期 传染 后期 会 更 惊讶 o 最新 疫情 地图 出炉 传播速度 惊讶


In [18]:
test_df = pd.read_csv('./processed_data/test.csv', encoding='utf-8')
test_df['processed_text'] = test_df['text'].apply(cut_and_clean)
test_df.head()

,text,label,processed_text
0,星火线上课 足不出户，同样进步！L星火教育呼市的 视频,1,星火 线 上课 足不出户 l 星火 教育 呼市 视频
1,中科院病毒研究所石正丽老师演讲《追踪SARS的源头》探索病毒与人类社会的关系。真是好看,1,中科院 病毒 研究所 石正丽 老师 演讲 追踪 sars 源头 探索 病毒 人类 社会 关系 好看
2,嗯 @熊中默就是那个给那位挂基层辅警照片 暴平安天门几万条艾滋病巨婴gay洗地还挂plmm照...,-1,@ 熊中默 那位 挂 基层 辅警 照片 暴 平安 天门 几万 条 艾滋病 巨婴 gay 洗地...
3,今天看到两条让我真的笑，笑出声的 。「为防病毒，出门抢购双黄连，染上病毒。」O 「新冠病毒感...,-1,两条 真的 笑 笑 出声 防病毒 出门 抢购 双黄连 染上 病毒 o 新冠 病毒感染 肾脏 ...
4,@_莫西干_,0,@ _ 莫西 干 _


In [41]:
train_data = train_df['processed_text']
train_label = train_df['label'].apply(lambda x: x+1).astype(int)

test_data = test_df['processed_text']
test_label = test_df['label'].apply(lambda x: x+1).astype(int)

In [57]:
vectorizer = CountVectorizer(max_df=5000, min_df=50)
tfidftransformer = TfidfTransformer()
pipe = Pipeline([('count', vectorizer),
                 ('tfid', tfidftransformer)])
pipe_model = pipe.fit(train_data)

train_preds = pipe_model.transform(train_data)
train_array = train_preds.toarray()

test_preds = pipe_model.transform(test_data)
test_array = test_preds.toarray()

In [58]:
train_array.shape

(79816, 4137)

In [59]:
dtrain = xgb.DMatrix(train_array, label=train_label)
dtest = xgb.DMatrix(test_array)

In [60]:
param = {'max_depth':10, 
         'eta':0.5, 
         'eval_metric':'merror', 
         'silent':1,
         'objective':'multi:softmax', 
         'num_class': 3}
evallist  = [(dtrain,'train')]
num_round = 30
bst = xgb.train(param, dtrain, num_round, evallist)

[17:45:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.29261
[1]	train-merror:0.27821
[2]	train-merror:0.27131
[3]	train-merror:0.26403
[4]	train-merror:0.25401
[5]	train-merror:0.24590
[6]	train-merror:0.23736
[7]	train-merror:0.23039
[8]	train-merror:0.22361
[9]	train-merror:0.21795
[10]	train-merror:0.21141
[11]	train-merror:0.20715
[12]	train-merror:0.20176
[13]	train-merror:0.19674
[14]	train-merror:0.19219
[15]	train-merror:0.18868
[16]	train-merror:0.18464
[17]	train-merror:0.18132
[18]	train-merror:0.17831
[19]	train-merror:0.17516
[20]	train-merror:0.17252
[21]	train-merror:0.16955
[22]	train-merror:0.16730
[23]	train-merror:0.16460
[24]	train

## 5. 模型结果

In [62]:
from sklearn.metrics import classification_report

In [63]:
preds = bst.predict(dtest)

In [64]:
result = classification_report(test_label, preds)
print(result)

              precision    recall  f1-score   support

           0       0.59      0.39      0.47      1796
           1       0.70      0.83      0.76      5653
           2       0.69      0.57      0.63      2551

    accuracy                           0.69     10000
   macro avg       0.66      0.60      0.62     10000
weighted avg       0.68      0.69      0.68     10000

